##### File containing the vessel definitions

Script modified from the work by Leonore Vehmeijer. Changes:
- Changed fieldnames (more consistent naming
- Added more class information (avv_class, cemt_class, type)

Sources:
- RWS, 2002. "Classificatie en kernmerken van de Europese vloot en de Actieve vloot in Nederland" Rijkswaterstaat, Adviesdienst Verkeer en Vervoer. 
- https://nl.wikipedia.org/wiki/CEMT-klasse

**Todo:**
- Thoroughly check the list and document where values came from
- Reconsider whether the loaded option should be included (doesn't seem very useful)
- Include 'velocity_loaded' en 'velocity_unloaded'


In [1]:
import pandas as pd
import numpy as np

In [11]:
# example M8
g = 9.81            # gravitational accelleration [m/s^2]
rho = 1025          # density water [kg/m3]

L = 106.95          # length of route [km]
B = 11.25           # width vessel [m]
T = 3.35            # maximum draught [m]

Tempty_gem = .8     # average draught of vessel [m]
Tempty_max = 1.4    # maximum draught of vessel [m]

Vfull = 4.59        # vessel speed full [m/s]
Veff = 4.767        # vessel speed effective [m/s]
Vempty = 5.14       # vessel speed empty [m/s]

nu = 1*10**(-6) 
Cp = 0.15
Cz = 0.2

W = 150             # channel width [m]
h = 2*T             # water depth [m]


##### Friction resistance (Rf)

In [13]:
Tmax = T
Tgem = T

Ac = W * h          # m2
Am = B * Tmax       # m2

V = Vfull

Fnh = V/np.sqrt(g * h)
print('Froude nr (Fn): {:.2f} [-]'.format(Fnh))

u = ((Am/Ac)/(1 - (Am/Ac) - Fnh**2))* V
print('Return current (u): {:.2f} m/s'.format(u))

Urel = np.max([V + u, Veff])
print('Relative speed (Urel): {:.2f} m/s'.format(Urel))

Rf = 53 * ((np.log(Urel * L) + 4)**(-2)) * (L * B + 2 * L * Tgem) * Urel**2
print('Friction resistance (Rf) full: {:.2f} N'.format(Rf))

Froude nr (Fn): 0.57 [-]
Return current (u): 0.27 m/s
Relative speed (Urel): 4.86 m/s
Friction resistance (Rf) full: 22843.11 N


In [128]:
Tgem = Tempty_gem
V = Vempty
# Tgem = T

h = 2*T
Fnh = V/np.sqrt(g * h)
print('Froude nr (Fn): {:.2f} [-]'.format(Fnh))

u = ((Am/Ac)/(1 - (Am/Ac) - Fnh**2))* V
print('Return current (u): {:.2f} m/s'.format(u))

Urel = np.max([V + u, Veff])
print('Relative speed (Urel): {:.2f} m/s'.format(Urel))

Rf = 53 * ((np.log(Urel * L) + 4)**(-2)) * (L * B + 2 * L * Tgem) * Urel**2
print('Friction resistance (Rf) empty: {:.2f} N'.format(Rf))



Froude nr (Fn): 0.63 [-]
Return current (u): 0.27 m/s
Relative speed (Urel): 5.41 m/s
Friction resistance (Rf) empty: 19834.75 N


##### Residual resistance (Rp)

In [129]:
V = Veff
Tmax = T
Rp = Cp * 0.5 * rho * V**2 * B * Tmax 
print('Residual resistance (Rp) full: {:.2f} N'.format(Rp))

Residual resistance (Rp) full: 65837.41 N


In [130]:
V = Vempty
Tmax = Tempty_max
Rp = Cp * 0.5 * rho * V**2 * B * Tmax 
print('Residual resistance (Rp) empty: {:.2f} N'.format(Rp))

Residual resistance (Rp) empty: 31988.36 N


##### Resistance due to limited waterdepth (Rz)

In [131]:
u = ((Am/Ac)/(1 - (Am/Ac) - Fnh**2))* Vfull
print('Return current (u): {:.2f} m/s'.format(u))

Return current (u): 0.24 m/s


In [132]:
z = ((Am/Ac)/(1 - (Am/Ac) - Fnh**2))* Fnh**2 * h
print('Water level depression (z): {:.2f} m'.format(z))

Water level depression (z): 0.14 m


In [134]:
Rz = Cz * rho * g * z * V**2 * B * T
print('Limited waterdepth (Rz): {:.2f} N'.format(Rz))

Limited waterdepth (Rz): 279245.67 N


##### Total resistance (Rf + Rp + Rz)

In [135]:
Rt = Rf + Rp + Rz
print('Total resistance (Rt): {:.2f} kN'.format(Rt))

Total resistance (Rt): 331068.77 kN


In [9]:
M8 = {
    'width': 11.4,                                 # RWS2002, 90%, significantie dm's
    'length': 110,                                 # RWS2002, 90%, significantie dm's
    'height_unloaded': 4,
    'height_loaded': 9.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.7,
    'installed_power': 318,
    'own_weight': 100,
    'capacity_loaded':2711*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.7777,
    'speed_unloaded': 5.3333,
    'resistance_loaded': 106453,
    'resistance_unloaded':61526,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M8",                             # RWS2002
    'cemt_class': "Va",                            # RWS2002
    'type': "Groot Rijnschip",                     # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M8_loaded = {
    'width': 11.4,
    'length':110,
    'height_unloaded': 4,
    'height_loaded': 9.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.7,
    'installed_power': 318,
    'own_weight': 100,
    'capacity_loaded':2711*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.7777,
    'speed_unloaded': 5.3333,
    'resistance_loaded': 106453,
    'resistance_unloaded':61526,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M8",                             # RWS2002
    'cemt_class': "Va",                            # RWS2002
    'type': "Groot Rijnschip",                     # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


M1 = {
    'width': 5.05,
    'length':38.5,
    'height_unloaded': 2,
    'height_loaded': 5.25,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.73,
    'installed_power': 36,
    'own_weight': 80,
    'capacity_loaded':400*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.083,
    'speed_unloaded': 4.47,
    'resistance_loaded': 16919,
    'resistance_unloaded':9188,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M1",                             # RWS2002
    'cemt_class': "I",                             # RWS2002
    'type': "Spits",                               # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M1_loaded = {
    'width': 5.05,
    'length':38.5,
    'height_unloaded': 2,
    'height_loaded': 5.25,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.73,
    'installed_power': 36,
    'own_weight': 80,
    'capacity_loaded':400*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.083,
    'speed_unloaded': 4.47,
    'resistance_loaded': 16919,
    'resistance_unloaded':9188,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M1",                             # RWS2002
    'cemt_class': "I",                             # RWS2002
    'type': "Spits",                               # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



M2 = {
    'width': 6.06,
    'length':55,
    'height_unloaded': 2,
    'height_loaded': 6.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.72,
    'installed_power': 118,
    'own_weight': 90,
    'capacity_loaded':650*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.361,
    'speed_unloaded': 4.638,
    'resistance_loaded': 28770,
    'resistance_unloaded':17524,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M2",                             # RWS2002
    'cemt_class': "II",                            # RWS2002
    'type': "Kempenaar",                           # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M2_loaded = {
    'width': 6.06,
    'length':55,
    'height_unloaded': 2,
    'height_loaded': 6.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.72,
    'installed_power': 118,
    'own_weight': 90,
    'capacity_loaded':650*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.361,
    'speed_unloaded': 4.638,
    'resistance_loaded': 28770,
    'resistance_unloaded':17524,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M2",                             # RWS2002
    'cemt_class': "II",                            # RWS2002
    'type': "Kempenaar",                           # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



M3 = {
    'width': 7.2,
    'length':70,
    'height_unloaded': 2,
    'height_loaded': 6.4,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.72,
    'installed_power': 120,
    'own_weight': 100,
    'capacity_loaded':800*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.861,
    'speed_unloaded': 4.638,
    'resistance_loaded': 43935,
    'resistance_unloaded':22746,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M3",                             # RWS2002
    'cemt_class': "III",                           # RWS2002
    'type': "Hagenaar",                            # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M3_loaded = {
    'width': 7.2,
    'length':70,
    'height_unloaded': 2,
    'height_loaded': 6.4,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.72,
    'installed_power': 120,
    'own_weight': 100,
    'capacity_loaded':800 * 0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.861,
    'speed_unloaded': 4.638,
    'resistance_loaded': 43935,
    'resistance_unloaded':22746,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M3",                             # RWS2002
    'cemt_class': "III",                           # RWS2002
    'type': "Hagenaar",                            # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}




M4 = {
    'width': 8.2,
    'length':73,
    'height_unloaded': 2,
    'height_loaded': 6.6,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.72,
    'installed_power': 160,
    'own_weight': 110,
    'capacity_loaded':1050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 4.000,
    'speed_unloaded': 4.889,
    'resistance_loaded': 50582,
    'resistance_unloaded':28777,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M4",                             # RWS2002
    'cemt_class': "III",                           # RWS2002
    'type': "Dortmund-Eems (L <= 74 m)",           # RWS2002
    "scenario": ["Base Case"],
}

M4_loaded = {
    'width': 8.2,
    'length':73,
    'height_unloaded': 2,
    'height_loaded': 6.6,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.72,
    'installed_power': 160,
    'own_weight': 110,
    'capacity_loaded':1050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 4.000,
    'speed_unloaded': 4.889,
    'resistance_loaded': 50582,
    'resistance_unloaded':28777,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M4",                             # RWS2002
    'cemt_class': "III",                           # RWS2002
    'type': "Dortmund-Eems (L <= 74 m)",           # RWS2002
    "scenario": ["Base Case"],
}


M5 = {
    'width': 8.2,
    'length':85,
    'height_unloaded': 2,
    'height_loaded': 6.4,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.72,
    'installed_power': 323,
    'own_weight': 140,
    'capacity_loaded':1250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.916,
    'speed_unloaded': 5.027,
    'resistance_loaded': 54096,
    'resistance_unloaded':31794,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M5",                             # RWS2002
    'cemt_class': "III",                           # RWS2002
    'type': "Verl. Dortmund (L > 74 m)",           # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M5_loaded = {
    'width': 8.2,
    'length':85,
    'height_unloaded': 2,
    'height_loaded': 6.4,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.72,
    'installed_power': 323,
    'own_weight': 140,
    'capacity_loaded':1250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.916,
    'speed_unloaded': 5.027,
    'resistance_loaded': 54096,
    'resistance_unloaded':31794,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M5",                             # RWS2002
    'cemt_class': "III",                           # RWS2002
    'type': "Verl. Dortmund (L > 74 m)",           # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}





M6 = {
    'width': 9.5,
    'length':85,
    'height_unloaded': 2,
    'height_loaded': 7,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 263,
    'own_weight': 120,
    'capacity_loaded':1750*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.916,
    'speed_unloaded': 5.194,
    'resistance_loaded': 70802,
    'resistance_unloaded':44676,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M6",                             # RWS2002
    'cemt_class': "IV",                            # RWS2002
    'type': "Rijn-Herne Schip (L <= 86 m)",        # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M6_loaded = {
    'width': 9.5,
    'length':85,
    'height_unloaded': 2,
    'height_loaded': 7,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 263,
    'own_weight': 120,
    'capacity_loaded':1750*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.916,
    'speed_unloaded': 5.194,
    'resistance_loaded': 70802,
    'resistance_unloaded':44676,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M6",                             # RWS2002
    'cemt_class': "IV",                            # RWS2002
    'type': "Rijn-Herne Schip (L <= 86 m)",        # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



M7 = {
    'width': 9.5,
    'length':105,
    'height_unloaded': 2,
    'height_loaded': 7,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 258,
    'own_weight': 110,
    'capacity_loaded':2050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.778,
    'speed_unloaded': 5.361,
    'resistance_loaded': 74294,
    'resistance_unloaded':53220,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M7",                             # RWS2002
    'cemt_class': "IV",                            # RWS2002
    'type': "Verl. Rijn-Herne Schip (L > 86 m)",   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M7_loaded = {
    'width': 9.5,
    'length':105,
    'height_unloaded': 2,
    'height_loaded': 7,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 258,
    'own_weight': 110,
    'capacity_loaded':2050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.778,
    'speed_unloaded': 5.361,
    'resistance_loaded': 74294,
    'resistance_unloaded':53220,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M7",                             # RWS2002
    'cemt_class': "IV",                            # RWS2002
    'type': "Verl. Rijn-Herne Schip (L > 86 m)",   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M9 = {
    'width': 11.4,
    'length':135,
    'height_unloaded': 3,
    'height_loaded': 9.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 350,
    'own_weight': 120,
    'capacity_loaded':4000*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.805,
    'speed_unloaded': 5.416,
    'resistance_loaded': 116161,
    'resistance_unloaded':69833,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M9",                             # https://nl.wikipedia.org/wiki/CEMT-klasse
    'cemt_class': "Va",                            # https://nl.wikipedia.org/wiki/CEMT-klasse
    'type': "Verl. Groot Rijnschip",               # https://nl.wikipedia.org/wiki/CEMT-klasse
    "scenario": ["NS-High", "NS-Low"],
}

M9_loaded = {
    'width': 11.4,
    'length':135,
    'height_unloaded': 3,
    'height_loaded': 9.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 350,
    'own_weight': 120,
    'capacity_loaded':4000*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.805,
    'speed_unloaded': 5.416,
    'resistance_loaded': 116161,
    'resistance_unloaded':69833,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M9",                             # https://nl.wikipedia.org/wiki/CEMT-klasse
    'cemt_class': "Va",                            # https://nl.wikipedia.org/wiki/CEMT-klasse
    'type': "Verl. Groot Rijnschip",               # https://nl.wikipedia.org/wiki/CEMT-klasse
    "scenario": ["NS-High", "NS-Low"],
}


M10 = {
    'width': 13.5,
    'length':135,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 350,
    'own_weight': 120,
    'capacity_loaded':4300*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.583,
    'speed_unloaded': 5.388,
    'resistance_loaded': 124101,
    'resistance_unloaded':72153,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M10",                            # https://nl.wikipedia.org/wiki/CEMT-klasse
    'cemt_class': "VIa",                           # https://nl.wikipedia.org/wiki/CEMT-klasse
    'type': "Maatgevend schip",                    # https://nl.wikipedia.org/wiki/CEMT-klasse
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M10_loaded = {
    'width': 13.5,
    'length':135,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 350,
    'own_weight': 120,
    'capacity_loaded':4300*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.583,
    'speed_unloaded': 5.388,
    'resistance_loaded': 124101,
    'resistance_unloaded':72153,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M10",                            # https://nl.wikipedia.org/wiki/CEMT-klasse
    'cemt_class': "VIa",                           # https://nl.wikipedia.org/wiki/CEMT-klasse
    'type': "Maatgevend schip",                    # https://nl.wikipedia.org/wiki/CEMT-klasse
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


M11 = {
    'width': 14.2,
    'length':135,
    'height_unloaded': 3,
    'height_loaded': 8,
    'emissionfactor': 0.70,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'installed_power': 380,
    'own_weight': 150,
    'capacity_loaded':5600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 4.027,
    'speed_unloaded': 5.277,
    'resistance_loaded': 141593,
    'resistance_unloaded':85947,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M11",                            # https://nl.wikipedia.org/wiki/CEMT-klasse
    'cemt_class': "VIa",                           # https://nl.wikipedia.org/wiki/CEMT-klasse
    'type': "Maatgevend schip",                    # https://nl.wikipedia.org/wiki/CEMT-klasse
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M11_loaded = {
    'width': 14.2,
    'length':135,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 380,
    'own_weight': 140,
    'capacity_loaded':5600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 4.027,
    'speed_unloaded': 5.277,
    'resistance_loaded': 141593,
    'resistance_unloaded':85947,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M11",                            # https://nl.wikipedia.org/wiki/CEMT-klasse
    'cemt_class': "VIa",                           # https://nl.wikipedia.org/wiki/CEMT-klasse
    'type': "Maatgevend schip",                    # https://nl.wikipedia.org/wiki/CEMT-klasse
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



M12 = {
    'width': 17,
    'length':135,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 380,
    'own_weight': 145,
    'capacity_loaded':5600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.888,
    'speed_unloaded': 5.138,
    'resistance_loaded': 167863,
    'resistance_unloaded':102290,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "M12",                            # https://nl.wikipedia.org/wiki/CEMT-klasse
    'cemt_class': "VIa",                           # https://nl.wikipedia.org/wiki/CEMT-klasse
    'type': "Rijnmax schip",                       # https://nl.wikipedia.org/wiki/CEMT-klasse
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

M12_loaded = {
    'width': 17,
    'length':135,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.70,
    'installed_power': 380,
    'own_weight': 145,
    'capacity_loaded':5600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.888,
    'speed_unloaded': 5.138,
    'resistance_loaded': 167863,
    'resistance_unloaded':102290,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "M12",                            # https://nl.wikipedia.org/wiki/CEMT-klasse
    'cemt_class': "VIa",                           # https://nl.wikipedia.org/wiki/CEMT-klasse
    'type': "Rijnmax schip",                       # https://nl.wikipedia.org/wiki/CEMT-klasse
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



BI = {
    'width': 9.5,
    'length':90,
    'height_unloaded': 3,
    'height_loaded': 7,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 263,
    'own_weight': 130,
    'capacity_loaded':1800*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.666,
    'speed_unloaded': 4.444,
    'resistance_loaded': 63671,
    'resistance_unloaded':27159,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BI",                             # RWS2002
    'cemt_class': "IV",                            # RWS2002
    'type': "1 baksduwstel Europa I",              # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


BI_loaded = {
    'width': 9.5,
    'length':90,
    'height_unloaded': 3,
    'height_loaded': 7,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 263,
    'own_weight': 130,
    'capacity_loaded':1800*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.666,
    'speed_unloaded': 4.444,
    'resistance_loaded': 63671,
    'resistance_unloaded':27159,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "BI",                             # RWS2002
    'cemt_class': "IV",                            # RWS2002
    'type': "1 baksduwstel Europa I",              # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}




BII_1 = {
    'width': 11.4,
    'length':110,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 318,
    'own_weight': 135,
    'capacity_loaded':3950*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.555,
    'speed_unloaded': 4.778,
    'resistance_loaded': 85750,
    'resistance_unloaded':37792,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_1",                          # RWS2002
    'cemt_class': "Va",                            # RWS2002
    'type': "1 baksduwstel Europa II",             # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

BII_1_loaded = {
    'width': 11.4,
    'length':110,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 318,
    'own_weight': 135,
    'capacity_loaded':3950*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.555,
    'speed_unloaded': 4.778,
    'resistance_loaded': 85750,
    'resistance_unloaded':37792,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "BII_1",                          # RWS2002
    'cemt_class': "Va",                            # RWS2002
    'type': "1 baksduwstel Europa II",             # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



BII_2b = {
    'width': 22.8,
    'length':145,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight': 150,
    'capacity_loaded':7050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.055,
    'speed_unloaded':4.722,
    'resistance_loaded': 140637,
    'resistance_unloaded':77029,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_2b",                          # RWS2002
    'cemt_class': "VIa",                            # RWS2002
    'type': "2 baksduwstel (breed)",                # RWS2002
    "scenario": ["NS-High"],
}

BII_2b_loaded = {
    'width': 22.8,
    'length':145,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight': 150,
    'capacity_loaded':7050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.055,
    'speed_unloaded':4.722,
    'resistance_loaded': 140637,
    'resistance_unloaded':77029,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "BII_2b",                          # RWS2002
    'cemt_class': "VIa",                            # RWS2002
    'type': "2 baksduwstel (breed)",                # RWS2002
    "scenario": ["NS-High"],
}


BII_2l = {
    'width': 11.4,
    'length':190,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight': 140,
    'capacity_loaded':7050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.638,
    'speed_unloaded':4.972,
    'resistance_loaded': 129810,
    'resistance_unloaded':55974,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_2l",                         # RWS2002
    'cemt_class': "Vb",                            # RWS2002
    'type': "2 baksduwstel (lang)",                # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

BII_2l_loaded = {
    'width': 11.4,
    'length':190,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight': 140,
    'capacity_loaded':7050*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.638,
    'speed_unloaded':4.972,
    'resistance_loaded': 129810,
    'resistance_unloaded':55974,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "BII_2l",                         # RWS2002
    'cemt_class': "Vb",                            # RWS2002
    'type': "2 baksduwstel (lang)",                # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


## Lower height for NS - Low scenario
BII_4 = {
    'width': 22.8,
    'length':195,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 400,
    'own_weight': 160,
    'capacity_loaded':12000*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 2.833,
    'speed_unloaded':4.611,
    'resistance_loaded': 861913,
    'resistance_unloaded':114953,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_4",                          # RWS2002
    'cemt_class': "VIb",                           # RWS2002
    'type': "4 baksduwstel",                       # RWS2002
    "scenario": ["Base Case", "NS-Low"],
}

BII_4_loaded = {
    'width': 22.8,
    'length':195,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 400,
    'own_weight': 160,
    'capacity_loaded':12000*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 2.833,
    'speed_unloaded':4.611,
    'resistance_loaded': 861913,
    'resistance_unloaded':114953,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "BII_4",                          # RWS2002
    'cemt_class': "VIb",                           # RWS2002
    'type': "4 baksduwstel",                       # RWS2002
    "scenario": ["Base Case", "NS-Low"],
}


BII_6l = {
    'width': 22.8,
    'length':270,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 400,
    'own_weight': 165,
    'capacity_loaded':18000*0.5,
    'capacity_unloaded':0,
    'speed_loaded': 2.667,
    'speed_unloaded':4.472,
    'resistance_loaded': 213665,
    'resistance_unloaded':151739,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "BII_6l",                         # RWS2002
    'cemt_class': "VIc",                           # RWS2002
    'type': "6 baksduwstel (lang)",                # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

BII_6l_loaded = {
    'width': 22.8,
    'length':270,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 400,
    'own_weight': 165,
    'capacity_loaded':18000*0.5,
    'capacity_unloaded':0,
    'speed_loaded': 2.667,
    'speed_unloaded':4.472,
    'resistance_loaded': 213665,
    'resistance_unloaded':151739,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "BII_6l",                         # RWS2002
    'cemt_class': "VIc",                           # RWS2002
    'type': "6 baksduwstel (lang)",                # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


B01 = {
    'width': 5.2,
    'length':55,
    'height_unloaded': 3,
    'height_loaded': 5.25,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 36,
    'own_weight': 80,
    'capacity_loaded':400*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.889,
    'speed_unloaded':4.167,
    'resistance_loaded': 27281,
    'resistance_unloaded':10473,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "B01",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwbak: 5.20 * 32.5",                 # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


B01_loaded = {
    'width': 5.2,
    'length':55,
    'height_unloaded': 3,
    'height_loaded': 5.25,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 36,
    'own_weight': 80,
    'capacity_loaded':400*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.889,
    'speed_unloaded':4.167,
    'resistance_loaded': 27281,
    'resistance_unloaded':10473,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "B01",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwbak: 5.20 * 32.5",                 # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



B02 = {
    'width': 6.6,
    'length':70,
    'height_unloaded': 3,
    'height_loaded': 6.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 118,
    'own_weight': 90,
    'capacity_loaded':600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.333,
    'speed_unloaded':4.305,
    'resistance_loaded': 36338,
    'resistance_unloaded':15187,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "B02",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwbak: 6.70 * 48",                   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


B02_loaded = {
    'width': 6.6,
    'length':70,
    'height_unloaded': 3,
    'height_loaded': 6.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 118,
    'own_weight': 90,
    'capacity_loaded':600*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.333,
    'speed_unloaded':4.305,
    'resistance_loaded': 36338,
    'resistance_unloaded':15187,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "B02",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwbak: 6.70 * 48",                   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



B03 = {
    'width': 7.5,
    'length':80,
    'height_unloaded': 3,
    'height_loaded': 6.4,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 120,
    'own_weight': 95,
    'capacity_loaded':800*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.667,
    'speed_unloaded':4.278,
    'resistance_loaded': 42647,
    'resistance_unloaded':18677,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "B03",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwbak: 7.50 * 55",                   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


B03_loaded = {
    'width': 7.5,
    'length':80,
    'height_unloaded': 3,
    'height_loaded': 6.4,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 120,
    'own_weight': 95,
    'capacity_loaded':800*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.667,
    'speed_unloaded':4.278,
    'resistance_loaded': 42647,
    'resistance_unloaded':18677,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "B03",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwbak: 7.50 * 55",                   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



B04 = {
    'width': 8.2,
    'length':85,
    'height_unloaded': 3,
    'height_loaded': 6.6,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 160,
    'own_weight': 95,
    'capacity_loaded':1250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.778,
    'speed_unloaded':4.361,
    'resistance_loaded': 47147,
    'resistance_unloaded':21147,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "B04",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwbak: 8.20 * 62",                   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


B04_loaded = {
    'width': 8.2,
    'length':85,
    'height_unloaded': 3,
    'height_loaded': 6.6,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 160,
    'own_weight': 95,
    'capacity_loaded':1250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.778,
    'speed_unloaded':4.361,
    'resistance_loaded': 47147,
    'resistance_unloaded':21147,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "B04",                            # RWS2002
    'cemt_class': "0",                             # RWS2002
    'type': "Duwbak: 8.20 * 62",                   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


C1b = {
    'width': 10.1,
    'length':38.5,
    'height_unloaded': 3,
    'height_loaded': 5.25,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 36,
    'own_weight': 80,
    'capacity_loaded':900*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.00,
    'speed_unloaded':4.222,
    'resistance_loaded': 28204,
    'resistance_unloaded':15387,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C1b",                            # RWS2002
    'cemt_class': "I",                             # RWS2002
    'type': "2 Spitsen breed",                     # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

C1b_loaded = {
    'width': 10.1,
    'length':38.5,
    'height_unloaded': 3,
    'height_loaded': 5.25,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 36,
    'own_weight': 80,
    'capacity_loaded':900*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.00,
    'speed_unloaded':4.222,
    'resistance_loaded': 28204,
    'resistance_unloaded':15387,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "C1b",                            # RWS2002
    'cemt_class': "I",                             # RWS2002
    'type': "2 Spitsen breed",                     # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}



C1l = {
    'width': 5.05,
    'length':80,
    'height_unloaded': 3,
    'height_loaded': 5.25,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 36,
    'own_weight': 80,
    'capacity_loaded':900*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.278,
    'speed_unloaded':4.389,
    'resistance_loaded': 20357,
    'resistance_unloaded':15211,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C1l",                            # RWS2002
    'cemt_class': "I",                             # RWS2002
    'type': "2 Spitsen lang",                      # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

C1l_loaded = {
    'width': 5.05,
    'length':80,
    'height_unloaded': 3,
    'height_loaded': 5.25,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 36,
    'own_weight': 80,
    'capacity_loaded':900*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.278,
    'speed_unloaded':4.389,
    'resistance_loaded': 20357,
    'resistance_unloaded':15211,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "C1l",                            # RWS2002
    'cemt_class': "I",                             # RWS2002
    'type': "2 Spitsen lang",                      # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


C2b = {
    'width': 19,
    'length':105,
    'height_unloaded': 3,
    'height_loaded': 9.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight':140,
    'capacity_loaded':3350*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 2.583,
    'speed_unloaded':4.167,
    'resistance_loaded': 93843,
    'resistance_unloaded':70473,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C2b",                            # RWS2002
    'cemt_class': "VIa",                           # RWS2002
    'type': "Klasse IV + 1 Europa I bak breed",    # RWS2002
    "scenario": ["NS-High", "NS-Low"],
}

C2b_loaded = {
    'width': 19,
    'length':105,
    'height_unloaded': 3,
    'height_loaded': 9.1,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight':140,
    'capacity_loaded':3350*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 2.583,
    'speed_unloaded':4.167,
    'resistance_loaded': 93843,
    'resistance_unloaded':70473,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "C2b",                            # RWS2002
    'cemt_class': "VIa",                           # RWS2002
    'type': "Klasse IV + 1 Europa I bak breed",    # RWS2002
    "scenario": [ "NS-High", "NS-Low"],
}



C2l = {
    'width': 9.5,
    'length':185,
    'height_unloaded': 3,
    'height_loaded': 7,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 258,
    'own_weight':140,
    'capacity_loaded':3350*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 3.583,
    'speed_unloaded':4.722,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C2l",                            # RWS2002
    'cemt_class': "IVb",                           # RWS2002
    'type': "Klasse IV + 1 Europa I bak lang",     # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

C2l_loaded = {
    'width': 9.5,
    'length':185,
    'height_unloaded': 3,
    'height_loaded': 7,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 258,
    'own_weight':140,
    'capacity_loaded':3350*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 3.583,
    'speed_unloaded':4.722,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "C2l",                            # RWS2002
    'cemt_class': "IVb",                           # RWS2002
    'type': "Klasse IV + 1 Europa I bak lang",     # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


C3b = {
    'width': 22.8,
    'length':110,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight':140,
    'capacity_loaded':7250*0.5,
    'capacity_unloaded':0,
    'speed_loaded': 2.333,
    'speed_unloaded':4.278,
    'resistance_loaded': 93843,
    'resistance_unloaded':70473,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C3b",                            # RWS2002
    'cemt_class': "VIa",                           # RWS2002
    'type': "Klasse Va + 1 Europa II bak breed",   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

C3b_loaded = {
    'width': 22.8,
    'length':110,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight':140,
    'capacity_loaded':7250*0.5,
    'capacity_unloaded':0,
    'speed_loaded': 2.333,
    'speed_unloaded':4.278,
    'resistance_loaded': 93843,
    'resistance_unloaded':70473,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "C3b",                            # RWS2002
    'cemt_class': "VIa",                           # RWS2002
    'type': "Klasse Va + 1 Europa II bak breed",   # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

C3l = {
    'width': 11.4,
    'length':190,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight':140,
    'capacity_loaded':7250*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 3.250,
    'speed_unloaded':5.056,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C3l",                            # RWS2002
    'cemt_class': "Vb",                            # RWS2002
    'type': "Klasse Va + 1 Europa II bak lang",    # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}

C3l_loaded = {
    'width': 11.4,
    'length':190,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 350,
    'own_weight':140,
    'capacity_loaded':7250*0.60,
    'capacity_unloaded':0,
    'speed_loaded': 3.250,
    'speed_unloaded':5.056,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "C3l",                            # RWS2002
    'cemt_class': "Vb",                            # RWS2002
    'type': "Klasse Va + 1 Europa II bak lang",    # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


C4 = {
    'width': 22.8,
    'length':185,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 400,
    'own_weight':150,
    'capacity_loaded':7250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.00,
    'speed_unloaded':4.472,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': False,
    'avv_class': "C4",                             # RWS2002
    'cemt_class': "VIb",                           # RWS2002
    'type': "Klasse Va + 3 Europa II bakken",      # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


C4_loaded = {
    'width': 22.8,
    'length':185,
    'height_unloaded': 3,
    'height_loaded': 8,
    'draught_unloaded': 1.5,                       # dummy values
    'draught_loaded': 2.7,                         # dummy values
    'emissionfactor': 0.71,
    'installed_power': 400,
    'own_weight':150,
    'capacity_loaded':7250*0.65,
    'capacity_unloaded':0,
    'speed_loaded': 3.00,
    'speed_unloaded':4.472,
    'resistance_loaded': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': True,
    'avv_class': "C4",                             # RWS2002
    'cemt_class': "VIb",                           # RWS2002
    'type': "Klasse Va + 3 Europa II bakken",      # RWS2002
    "scenario": ["Base Case", "NS-High", "NS-Low"],
}


ships = [M1, M1_loaded, 
         M2, M2_loaded, 
         M3, M3_loaded, 
         M4, M4_loaded, 
         M5, M5_loaded, 
         M6, M6_loaded, 
         M7, M7_loaded, 
         M8, M8_loaded,
         M9, M9_loaded, 
         M10, M10_loaded, 
         M11, M11_loaded, 
         M12, M12_loaded, 
         BI, BI_loaded, 
         BII_1, BII_1_loaded, 
         BII_2l, BII_2l_loaded,
         BII_2b, BII_2b_loaded,
         BII_4, BII_4_loaded, 
         B01, B01_loaded, 
         B02, B02_loaded, 
         B03, B03_loaded, 
         B04, B04_loaded,
         C1l, C1l_loaded, 
         C1b, C1b_loaded, 
         C2l, C2l_loaded, 
         C2b, C2b_loaded, 
         C3l, C3l_loaded, 
         C3b, C3b_loaded, 
         C4, C4_loaded,
         BII_6l, BII_6l_loaded]

In [10]:
import uuid
str(uuid.uuid1())

'c607308c-9b57-11ea-aedc-60f677ba2ce0'

In [11]:
ship_id = ["29a24858-4a51-11e9-9792-b469212bff5b", "3254257e-4a51-11e9-8548-b469212bff5b",
           "dcc05ff4-4972-11e9-a543-b469212bff5b", "dcc05ff4-4972-11e9-a543-b469212bff5b", 
           "e6cea50a-4972-11e9-a1f1-b469212bff5b", "e6cea50a-4972-11e9-a1f1-b469212bff5b",
           "f0eb9740-4972-11e9-96ab-b469212bff5b", "f0eb9740-4972-11e9-96ab-b469212bff5b", 
           "f5685128-4972-11e9-9bc8-b469212bff5b", "f5685128-4972-11e9-9bc8-b469212bff5b", 
           "fa18f668-4972-11e9-be98-b469212bff5b", "fa18f668-4972-11e9-be98-b469212bff5b", 
           "fec15af4-4972-11e9-97be-b469212bff5b", "fec15af4-4972-11e9-97be-b469212bff5b", 
           "045a2bc2-4973-11e9-ac77-b469212bff5b", "045a2bc2-4973-11e9-ac77-b469212bff5b", 
           "0cbee714-4973-11e9-a6c3-b469212bff5b", "0cbee714-4973-11e9-a6c3-b469212bff5b", 
           "11069770-4973-11e9-8083-b469212bff5b", "11069770-4973-11e9-8083-b469212bff5b", 
           "147059ae-4973-11e9-8d83-b469212bff5b", "147059ae-4973-11e9-8d83-b469212bff5b", 
           "1827f876-4973-11e9-84b9-b469212bff5b", "1827f876-4973-11e9-84b9-b469212bff5b", 
           "1be8986e-4973-11e9-baae-b469212bff5b", "1be8986e-4973-11e9-baae-b469212bff5b", 
           "24bb5f98-4973-11e9-be88-b469212bff5b", "24bb5f98-4973-11e9-be88-b469212bff5b", 
           "28c5310a-4973-11e9-938b-b469212bff5b", "28c5310a-4973-11e9-938b-b469212bff5b", 
           "3027cb64-4973-11e9-b276-b469212bff5b", "3027cb64-4973-11e9-b276-b469212bff5b", 
           "34138582-4973-11e9-9237-b469212bff5b", "34138582-4973-11e9-9237-b469212bff5b", 
           "37eb9770-4973-11e9-87fe-b469212bff5b", "37eb9770-4973-11e9-87fe-b469212bff5b", 
           "3c8896ac-4973-11e9-8cff-b469212bff5b", "3c8896ac-4973-11e9-8cff-b469212bff5b", 
           "4073635c-4973-11e9-b613-b469212bff5b", "4073635c-4973-11e9-b613-b469212bff5b", 
           "463fde30-4973-11e9-ba75-b469212bff5b", "463fde30-4973-11e9-ba75-b469212bff5b",
           "49c3124a-4973-11e9-be8c-b469212bff5b", "49c3124a-4973-11e9-be8c-b469212bff5b", 
           "4db2931a-4973-11e9-ae43-b469212bff5b", "4db2931a-4973-11e9-ae43-b469212bff5b", 
           "5200e342-4973-11e9-b821-b469212bff5b", "5200e342-4973-11e9-b821-b469212bff5b", 
           "560257d0-4973-11e9-be15-b469212bff5b", "560257d0-4973-11e9-be15-b469212bff5b", 
           "5972634c-4973-11e9-bd78-b469212bff5b", "5972634c-4973-11e9-bd78-b469212bff5b", 
           "60f4dfa4-4973-11e9-9a85-b469212bff5b", "60f4dfa4-4973-11e9-9a85-b469212bff5b", 
           "667d1536-4973-11e9-a5bb-b469212bff5b", "667d1536-4973-11e9-a5bb-b469212bff5b",
           "6a9cbdee-4973-11e9-84e3-b469212bff5b", "6a9cbdee-4973-11e9-84e3-b469212bff5b"]

In [12]:
vessel_db = pd.DataFrame.from_dict({'VesselID': [],
                                    'width': [],
                                    'length': [],
                                    'height_unloaded': [],
                                    'height_loaded': [],
                                    'draught_unloaded': [],
                                    'draught_loaded': [],  
                                    'emissionfactor': [],
                                    'installed_power': [],
                                    'own_weight': [],
                                    'capacity_loaded': [],
                                    'capacity_unloaded': [],
                                    'speed_loaded': [],
                                    'speed_unloaded': [],
                                    'resistance_loaded': [],
                                    'resistance_unloaded': [],
                                    'is_loaded': [],
                                    'avv_class': [],
                                    'cemt_class': [],
                                    'type': [],
                                    'scenario': []})

for i, ship in enumerate(ships):
    ship["VesselID"] = ship_id[i]
    vessel_db = vessel_db.append(ship, ignore_index = True)

vessel_db.head()
vessel_db.to_csv("Vessel-database.csv", index = False)